# **RAG_Uberhtml_Chatbot_with_llamaindex_create_multiple_index**

- Here we are reading UBER HTML Files, which are in zip format with 4 years html 4 files
- Then apply **llamaindex- Vector Index** technique, each year html will have 1 separate index
- Then built **Retrival** on this vectorindex and fed to LLM Model for retrival.
- In this approch no other Vectordb used other than **vector index**



# **WAY 1: Pure Llamaindex and OpenAI agent used**

- Import all pacakges and Connect to OpenaAI

- Load the **Uber HTML files** data from website
	- Unzip the files and keep that file inside - **/content/data folder**
  - Sample html file : file:///D:/Prabha/Data%20Science/Prabha-DS/Gen_AI/Ineuron/Gen_AI_Course/Project/github/Gen_AI_Projects/07_RAG/05_Llamaindex_Tutorial/01_RAG_Uberhtml_Chatbot_with_llamaindex_create_multiple_index/Images/UBER_2019.html

- Read **HTML file** from **Llamaindex UnstructuredReader**
	- Start read the html and save doc in year wise Dictionary
- Apply **Embedding and create VectorIndex** by **Llamaindex properties**.
	- Here no other vectordb used
	- Store/Set all parameter/Config during run time to create vector index using llamaindex **Settings** option
	- For **Each year create the Index:** Here Index is nothing but Vector Database. Created this index inside **storage folder yearwise**
		- Similarly we can **create multiple index** regarding **multiple data** or **multiple files** or **multiple year**

- Retrieval From **In-memory Llamaindex Vectorestore** using **QueryEngineTool class**
	- https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/tools/query_engine.py

	- Here we can create query engine in below ways:

		- **1. Create yearwise Individual query engine - individual_query_engine_tools**
			- Here **every year is a individual query_engin**

		- **2. Create Entire query engine - query_engine tool**
			- Created **combined query_engine** using **individual_query_engine_tools**

		- **3. Combine Individual Query engine and Entire query engine**

		- **4. Create Final Agent: Use OpenAIAgent and pass combined query engine-tools**

			- Here **OpenAI Agent** and **query engine*** combination helps to **keep the memory of conversion** and provides response
			- Here Agents manages history also

--------------------------------------------------------

# WAY 2: Create Multiple Index on multiple PDF using Llamaindex retriver advanced properties

- In this example previosly created **Llama index vector/Index** are used. Not executed same step agin here
- But this script **not working** due to LLMPredictor not imported, so commented all


--------------------------------------------------------

# WAY 3: Combined approcah using llamaindex and langchain

- Here **llama index -  VectorStoreIndex,SimpleDirectoryReader** features used and **langchain - agent/Tool, ConversationBufferMemory, ChatOpenAI** features used

- Create data2 folder in colab inmemory and upload some pdf. Uploaded some pdf manually to this data2 folder

- Create Direct index on document/pdf using **Llamaindex VectorStoreIndex**(Here no chunking/embedding done, automaticaly it created embedding using default llm embedding model)

- Create langchain Tool structure.
	- **LlamaIndex retriver/as_query_engine from index** also used inside tool

- Mention the agent name and add langchains **ConversationBufferMemory** features
- Initialize **agent which is the final AGENT** by passing **llm(ChatOpenAI), tools,memory** detail
- Test the final agent by passing user Q

# Folder structure

  ![plot](Images/1_HTML_File.png)
  ![plot](Images/2_Inmemory_Vectore_Index.png)



# **Import all pacakges**

In [1]:
%pip install llama-index-readers-file llama-index-llms-openai  llama-index-embeddings-openai llama-index-agent-openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.6 MB/s eta 0:00:00


### **Connect to OpenaAI**
- But later we used hugging face, as there was token limit crossed in OpenAI :(

In [2]:
import os
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

# **Load the Uber HTML file data from website**
- Unzip the files and keep that file inside - **/content/data** folder

In [3]:
!mkdir data
#Create blank data folder

In [6]:
!wget "https://www.dropbox.com/s/948jr9cfs7fgj99/UBER.zip?dl=1" -O data/UBER.zip
#Read UBER.zip file from above link and save inside data folder

--2024-05-07 13:39:51--  https://www.dropbox.com/s/948jr9cfs7fgj99/UBER.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/948jr9cfs7fgj99/UBER.zip [following]
--2024-05-07 13:39:51--  https://www.dropbox.com/s/dl/948jr9cfs7fgj99/UBER.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8c8868e2824cfd8f2b71d8ac0d.dl.dropboxusercontent.com/cd/0/get/CSddsIAVBxpVH5nZE1uYd1oDms5bPsaMmB31i3eBlbFQhup8Hyozp7_OMJSu5EsJMo2LA5tK6YDHZT8wr3sIL12M2vQ8zFHWM6pVmgasp2WxWs894XGghbIEo9W8NzetVIsNKmGxBbo95tcQympDMVO3/file?dl=1# [following]
--2024-05-07 13:39:51--  https://uc8c8868e2824cfd8f2b71d8ac0d.dl.dropboxusercontent.com/cd/0/get/CSddsIAVBxpVH5nZE1uYd1oDms5bPsaMmB31i3eBlbFQhup8Hyozp7_OMJSu5EsJMo2LA5tK6YDHZT8wr3sIL12M2vQ8zFHWM6pVmgasp2WxWs89

In [7]:
!unzip data/UBER.zip -d data
#Unzip that file inside Local data folder

Archive:  data/UBER.zip
   creating: data/UBER/
  inflating: data/UBER/UBER_2021.html  
  inflating: data/__MACOSX/UBER/._UBER_2021.html  
  inflating: data/UBER/UBER_2020.html  
  inflating: data/__MACOSX/UBER/._UBER_2020.html  
  inflating: data/UBER/UBER_2019.html  
  inflating: data/__MACOSX/UBER/._UBER_2019.html  
  inflating: data/UBER/UBER_2022.html  
  inflating: data/__MACOSX/UBER/._UBER_2022.html  


# **Read HTML file from Llamaindex UnstructuredReader**

In [8]:
from llama_index.readers.file import UnstructuredReader #To read HTML File
from pathlib import Path
import nltk

#download this averaged_perceptron_tagger package to support UnstructuredReader()
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

### **Start read the html and save doc in year wise Dictionary**

In [9]:
!pip install unstructured -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 49.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 6.9 MB/s eta 0:00:00


In [10]:
years=[2019,2020,2021,2022] #Yearwise HTML file

#Create 1 html content by looping through 4 files
all_doc=[]
doc_set={}

#Create UnstructuredReader object
loader=UnstructuredReader()

In [11]:
#loader.load_data(file=Path(f"/content/data/UBER/UBER_2019.html"))

In [11]:
for year in years:
  print(f"/content/data/UBER/UBER_{year}.html")

  #Read the each HTML file
  year_doc=loader.load_data(file=Path(f"/content/data/UBER/UBER_{year}.html"))

  for doc in year_doc:
    doc.metadata={"year":year}  #Read the metdata

  doc_set[year]=year_doc  #Append Metdata
  all_doc.extend(year_doc) #Append all data. Yeardoc is a list, jsut extended

/content/data/UBER/UBER_2019.html
/content/data/UBER/UBER_2020.html
/content/data/UBER/UBER_2021.html
/content/data/UBER/UBER_2022.html


In [ ]:
#doc_set[2019]

In [ ]:
#doc_set[2022]

In [ ]:
#all_doc

# **Apply Embedding and create VectorIndex by Llamaindex properties**

In [12]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings  # This Settings required to set all hyperparameter value

## **Store/Set all parameter/Config during run time**

In [13]:
!pip install llama-index-embeddings-huggingface llama-index-embeddings-instructor -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.9 MB/s eta 0:00:00


In [14]:
#Either this(Settings) way or we can use # ServiceContext: this is to set parameter. # It in new version
Settings.chunk_size=512
Settings.chunk_overlap=64
Settings.llm=OpenAI(model="gpt-3.5-turbo")
#Settings.embed_model=OpenAIEmbedding(model="text-embedding-3-small") # OpenAI token limit crossed, so using Hugging face embedding

# Import embedding model from hugging face
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
Settings.embed_model=HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")


#OR

#Service_Context=ServiceContext.from_defaults(
#    chunk_size=1024,
#   llm=llm, #Mistral model
#    embed_model=embed_model #define embedding model
#)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## **For Each year create the Index**
- Here Index is nothing but **Vector Database**. Created this index inside **storage folder yearwise**

-  **Similarly we can create multiple index** regarding **multiple data or multiple files or multiple year**

In [15]:
!mkdir storage

In [16]:
index_set={}  #Blank Index

for year in years:


  storage_context=StorageContext.from_defaults() # This used to set any other External Vectordb, if not mentioned, it takes default

  # All parameter which we set above under Settings, will be passed here automatically during index creation
  # Ex:  Settings.chunk_size, Settings.chunk_overlap, Settings.llm, Settings.embed_model

  # Create index - which saved in in-memory vectore store by llamaindex
  index=VectorStoreIndex.from_documents(
                                doc_set[year],
                                storage_context=storage_context,
                                # embed_model : Here default it takes OpenAIEmbedding which set above
                                 )
  #Create index - which saved in in-memory vectore store by llamaindex
  #index = MultiModalVectorStoreIndex.from_documents(documents,
  #                                                  service_context=Service_Context,  #this is to set Chunk size etc parameter
  #                                                  storage_context=storage_context, # This used to set External Vectordb
  #                                                  embed_model =embed_model)

  #Save yearwise index inside Dictionary ; {Year:Index}
  index_set[year]=index

  #Save/persist the index in local folder
  storage_context.persist(persist_dir=f"/content/storage/{year}")

# **Retrieval From In-memory Llamaindex Vectorestore using query_engine class**

- https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/tools/query_engine.py


In [17]:
! pip install llama-index-question-gen-openai -q

In [18]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine

## **1. Create yearwise Individual query engine - individual_query_engine_tools**

In [19]:
individual_query_engine_tools = [
    QueryEngineTool(
        #Here every year is a individual query_engine
        query_engine=index_set[year].as_query_engine(), #Pass the index/ index_set[year]
        metadata=ToolMetadata(
            name=f"vector_index_{year}",
            description=(
                "useful for when you want to answer queries about the"
                f" {year} for Uber"
                        ),
                      ),
    )
    for year in years  # This helps to loop over each year and create list of individual_query_engine_tools
]

In [20]:
individual_query_engine_tools

## **2. Create Entire query engine - query_engine tool**

In [21]:
#Create query_engine using individual_query_engine_tools
query_engine = SubQuestionQueryEngine.from_defaults(
               query_engine_tools=individual_query_engine_tools,
              )

query_engine_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="sub_question_query_engine",
        description=(
            "useful for when you want to answer queries that require analyzing"
            " multiple SEC 10-K documents for Uber"
        ),
    ),
)

In [22]:
query_engine_tool

## **3. Combine Individual Query engine and Entire query engine**

In [23]:
tools = individual_query_engine_tools + [query_engine_tool]

## **4. Create Final Agent: Use OpenAIAgent and pass combined query engine-tools**

- Here **OpenAI Agent** and **query engine** combination helps to keep the **memory of conversion** and provides **response**
- Here **Agents** manages history also

In [24]:
from llama_index.agent.openai import OpenAIAgent

In [25]:
 agent=OpenAIAgent.from_tools(tools,verbose=True) #Pass combined query engine-tools

In [26]:
response=agent.chat("hi i am sunny")

Added user message to memory: hi i am sunny


In [27]:
print(str(response))

Hello Sunny! How can I assist you today?


In [28]:
response=agent.chat("what were some risk factor in 2020 with uber?")

Added user message to memory: what were some risk factor in 2020 with uber?
=== Calling Function ===
Calling function: vector_index_2020 with args: {"input":"risk factors"}
Got output: Some of the risk factors outlined in the context information include the adverse effects of the COVID-19 pandemic, potential reclassification of drivers as employees, intense competition in the mobility industry, significant losses incurred, challenges in attracting and maintaining platform users, operational and cultural challenges, negative media coverage affecting brand reputation, difficulties in managing growth, safety incidents impacting user retention, and risks associated with substantial investments in new technologies.



In [29]:
print(str(response))

In 2020, some of the risk factors for Uber included the adverse effects of the COVID-19 pandemic, potential reclassification of drivers as employees, intense competition in the mobility industry, significant losses, challenges in attracting and maintaining platform users, operational and cultural challenges, negative media coverage affecting brand reputation, difficulties in managing growth, safety incidents impacting user retention, and risks associated with substantial investments in new technologies.


In [30]:
response=agent.chat("what were some good factor in 2020 with uber?")

Added user message to memory: what were some good factor in 2020 with uber?
=== Calling Function ===
Calling function: vector_index_2020 with args: {"input":"positive factors"}
Got output: Equity incentives to attract and retain key personnel, rapid growth in the United States and internationally, and the implementation of reductions in workforce to optimize organizational structure.



In [31]:
print(str(response))

Some of the positive factors for Uber in 2020 included equity incentives to attract and retain key personnel, rapid growth in the United States and internationally, and the implementation of reductions in workforce to optimize organizational structure.


In [32]:
response=agent.chat("i didn't get those risk factor can you explain me 3rd and 4th point")

Added user message to memory: i didn't get those risk factor can you explain me 3rd and 4th point
=== Calling Function ===
Calling function: sub_question_query_engine with args: {"input": "In 2020, what were the challenges in attracting and maintaining platform users for Uber?"}
Generated 2 sub questions.
Got output: Error: asyncio.run() cannot be called from a running event loop

=== Calling Function ===
Calling function: sub_question_query_engine with args: {"input": "In 2020, what were the operational and cultural challenges faced by Uber?"}
Generated 2 sub questions.
Got output: Error: asyncio.run() cannot be called from a running event loop



In [33]:
print(str(response))

It seems there was an issue retrieving the specific details for the 3rd and 4th points regarding the challenges in attracting and maintaining platform users and the operational and cultural challenges faced by Uber in 2020. Let me try to provide a summary based on the general knowledge.

In 2020, Uber faced challenges in attracting and maintaining platform users due to intense competition in the mobility industry. The company had to continuously innovate and offer competitive services to retain existing users and attract new ones.

Regarding operational and cultural challenges, Uber may have encountered difficulties in managing its operations efficiently and aligning its organizational culture with its business goals. This could include issues related to internal processes, communication, and adapting to changes in the market.

For more detailed information, I recommend referring to Uber's official reports or statements for a comprehensive understanding of these challenges in 2020.


In [34]:
response=agent.chat("what is addition of 2+2?")
print(str(response))

Added user message to memory: what is addition of 2+2?
The addition of 2 + 2 is 4.


In [35]:
response=agent.chat("what was the revenu of uber in 2022?")
print(str(response))

Added user message to memory: what was the revenu of uber in 2022?
=== Calling Function ===
Calling function: vector_index_2022 with args: {"input":"revenue"}
Got output: Revenue increased by $14.4 billion in 2022 compared to 2021, primarily driven by a rise in Gross Bookings of 28%, or 33% on a constant currency basis. The growth in revenue was mainly due to increased Mobility Trip volumes as the business recovered from the impacts of COVID-19, a $4.8 billion increase in Freight Gross Bookings following the acquisition of Transplace, and a $3.9 billion net increase in Mobility revenue due to business model changes in the UK and accruals made for the resolution of historical claims in the UK related to the classification of drivers.

In 2022, Uber's revenue increased by $14.4 billion compared to 2021. This growth was primarily driven by a rise in Gross Bookings of 28%, or 33% on a constant currency basis. The increase in revenue was attributed to factors such as increased Mobility Trip

In [36]:
response=agent.chat("can you multiply those previous number?")
print(str(response))

Added user message to memory: can you multiply those previous number?
To multiply the revenue increase of $14.4 billion in 2022 by the rise in Gross Bookings of 28%, we would calculate:

$14.4 billion * 28% = $4.032 billion

Therefore, the result of multiplying the revenue increase by the rise in Gross Bookings is approximately $4.032 billion.


> Respone: The multiplication of $14.4 billion by 14.4 billion is $205,920 billion. Additionally, the multiplication of 28 by 33 is 924.


In [37]:
response=agent.chat("can you multiply those previous to previous number?")
print(str(response))

Added user message to memory: can you multiply those previous to previous number?
I apologize for the confusion, but I do not have access to the specific numbers mentioned in the previous to previous response. If you provide me with the numbers you are referring to, I would be happy to assist you with the multiplication calculation.


In [38]:
response=agent.chat("can you multiply previos two number which you added?")
print(str(response))

Added user message to memory: can you multiply previos two number which you added?
To multiply the revenue increase of $14.4 billion in 2022 by the rise in Gross Bookings of 28%, we calculated the result to be approximately $4.032 billion. 

Now, let's multiply this result by the constant currency basis increase of 33%:

$4.032 billion * 33% = $1.33056 billion

Therefore, the result of multiplying the previous two numbers (revenue increase by 28% and then by 33%) is approximately $1.33056 billion.


> It supposed to remember previous Q and ans, but its not doing :(

# **WAY 2: Create Multiple Index on multiple PDF using Llamaindex retriver advanced properties**

- In this example previosly created **Llama index vector/Index** are used. Not executed same step agin here

- But this script not working due to LLMPredictor not imported, so commented all

In [46]:
# !pip install langchain llama_index -q
# !pip install llama-index-llms-openai -q
# !pip install llama-index-llms-langchain -q

In [ ]:
# from llama_index.core import SimpleDirectoryReader
# from llama_index.core import  ServiceContext, VectorStoreIndex
# #from llama_index.LLMPredictor import LLMPredictor
# from llama_index.core.response.pprint_utils import pprint_response
# from langchain import OpenAI

# from llama_index.core.tools import QueryEngineTool, ToolMetadata
# from llama_index.core.query_engine import SubQuestionQueryEngine

# # Initiated LLM/ model_name="text-davinci-003"
# llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003", max_tokens=-1, streaming=True))

# #Define this LLM model as ServiceContext
# service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)


# march_2022 = SimpleDirectoryReader(input_files=["llama_index/docs/examples/data/10q/uber_10q_march_2022.pdf"]).load_data()
# june_2022 = SimpleDirectoryReader(input_files=["llama_index/docs/examples/data/10q/uber_10q_june_2022.pdf"]).load_data()
# sept_2022 = SimpleDirectoryReader(input_files=["llama_index/docs/examples/data/10q/uber_10q_sept_2022.pdf"]).load_data()

# march_index = VectorStoreIndex.from_documents(march_2022)
# june_index = VectorStoreIndex.from_documents(june_2022)
# sept_index = VectorStoreIndex.from_documents(sept_2022)

# march_engine = march_index.as_query_engine(similarity_top_k=3)
# june_engine = june_index.as_query_engine(similarity_top_k=3)
# sept_engine = sept_index.as_query_engine(similarity_top_k=3)

# query_engine_tools = [
#     QueryEngineTool(
#         query_engine=sept_engine,
#         metadata=ToolMetadata(name='sept_22', description='Provides information about Uber quarterly financials ending September 2022')
#     ),
#     QueryEngineTool(
#         query_engine=june_engine,
#         metadata=ToolMetadata(name='june_22', description='Provides information about Uber quarterly financials ending June 2022')
#     ),
#     QueryEngineTool(
#         query_engine=march_engine,
#         metadata=ToolMetadata(name='march_22', description='Provides information about Uber quarterly financials ending March 2022')
#     ),
# ]

# # Given a query, this query engine `SubQuestionQueryEngine ` will generate a “query plan”
# # containing sub-queries against sub-documents before synthesizing the final answer.
# s_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)

# response = s_engine.query('Analyze Uber revenue growth over the latest two quarter filings')

# print(response)

# **WAY 3: Combined approcah using llamaindex and langchain**
- Here **llama index -  VectorStoreIndex,SimpleDirectoryReader** features used and **langchain - agent/Tool, ConversationBufferMemory, ChatOpenAI** features used

In [54]:
!pip install langchain -q

In [55]:
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

In [56]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader

### Create data2 folder in colab inmemory and upload some pdf

In [57]:
!mkdir data2

In [58]:
document=SimpleDirectoryReader('/content/data2').load_data()

### Uploaded some pdf manually to this data2 folder

## Create Direct index on document/pdf using **Llamaindex VectorStoreIndex** (Here no chunking/embedding done, automaticaly it created embedding using default llm embedding model)

In [59]:
#LlamaIndex - index
index = VectorStoreIndex.from_documents(documents=document)

## Create langchain Tool structure.
- **LlamaIndex retriver/as_query_engine from index** also used inside tool

In [60]:
tools=[
    Tool(
        # Use Langchain class - Tool
        name = "LlamaIndex",  #Name of the Tool
        func=lambda q: str(index.as_query_engine().query(q)),  #LlamaIndex retriver/as_query_engine from index
        description="useful for when you want to answer questions about the author. The input to this tool should be a complete english sentence.",
        return_direct=True
    ),

]

## Mention the agent name and add langchains **ConversationBufferMemory** features

In [61]:
#Mention the agent name
agent="conversational-react-description"

In [62]:
memory=ConversationBufferMemory(memory_key = "chat_history")

## Initialize **agent which is the final AGENT** by passing **llm(ChatOpenAI), tools,memory** detail

In [63]:
#Create LLM -  Used Openai gpt
llm = ChatOpenAI(temperature=0)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [64]:
# initialize agent which is the final AGENT
agent_executor=initialize_agent(tools,llm,agent,memory=memory)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


## Test the final agent by passing user Q

In [65]:
agent_executor.run(input="hi i am sunny")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'Hello Sunny! How can I assist you today?'

In [66]:
agent_executor.run(input="what is RAG token?")

'RAG-Token can be viewed as a standard, autoregressive seq2seq generator with a specific transition probability.'

In [67]:
agent_executor.run(input="what will be the sum of 2 and 2?")

'The sum of 2 and 2 is 4.'

In [68]:
agent_executor.run(input="what is RAG sequence model?")

'The RAG-Sequence model does not break the likelihood into a conventional per-token likelihood, which means it cannot be solved with a single beam search. Instead, beam search is run for each document z, scoring each hypothesis using pθ(yi|x,z,y1:i-1). This results in a set of hypotheses Y, some of which may not have appeared in the beams of all documents. To estimate the probability of a hypothesis y, an additional forward pass is run for each document z for which y does not appear in the beam, multiplying the generator probability with pη(z|x) and then summing the probabilities across beams for the marginals. This decoding procedure is referred to as "Thorough Decoding."'

In [69]:
agent_executor.run(input="can you multiply those previous two numbers?")

'16'

In [70]:
agent_executor.run(input="what is latest rag evaltion technique in 2024?")

"The latest RAG evaluation technique in 2024 involves conducting ablations on the dev set to compare different RAG models. The evaluation includes metrics such as Exact Match, Label Accuracy, and performance with mismatched indices to update RAG's world knowledge by replacing its non-parametric memory. Additionally, the evaluation considers the effect of retrieving more documents on RAG models' performance, with the flexibility to adjust the number of retrieved documents at test time for improved results."

In [71]:
agent_executor.run(input="what is llama3 model and is it appropriate for RAG?")

'The llama3 model is not mentioned in the provided context information.'

In [72]:
agent_executor.run(input="what is machine learning?")

'Machine learning is a field of artificial intelligence that focuses on the development of algorithms and models that enable computers to learn from and make predictions or decisions based on data, without being explicitly programmed to do so.'

In [73]:
agent_executor.run(input="what is 10+10?")

'The sum of 10 and 10 is 20.'

In [ ]:
agent_executor.run(input="can you tell me the types of it?")

# **END**